In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import tqdm
import sys

In [15]:
def read_csv(directory, folder, filename):
    df = pd.read_csv(directory + folder + filename)
    df = df.replace(r'^\s*$', np.nan, regex=True) #replaces empty strings spacess with NaN
    return df

In [3]:
directory = '/Volumes/GoogleDrive-114506167238584938104/My Drive/common_test_files/'
folder = 'bug_exp/'

df = read_csv(directory, folder, 'train-hrt-30%-NaN.csv')



In [9]:
valid_data_types = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', 'bool']

valid = df.select_dtypes(include=valid_data_types)

col_is_numeric_cat = []

for col in valid.columns:

    unique = valid[col].unique()
    unique = unique[~np.isnan(unique)] #remove nan
    unique_int_len = len(list(filter(lambda x: x.is_integer(), unique))) #checks to ensure values are ints even if cast as float

    if len(unique) == unique_int_len and (unique_int_len > 2 and unique_int_len < 6): #set 
        col_is_numeric_cat.append(col)

print(col_is_numeric_cat)




['restecg', 'slope', 'ca', 'thal']


In [18]:
df.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'HeartDz'],
      dtype='object')

In [23]:
imp_mean = IterativeImputer(random_state=0)
X = df[['restecg', 'slope', 'ca', 'thal']]
imp_mean.fit(X)
result = pd.DataFrame(imp_mean.transform(X),columns=X.columns)
result

,restecg,slope,ca,thal
0,0.554468,1.445465,0.00000,2.322317
1,1.000000,0.000000,0.00000,2.495858
2,0.000000,2.000000,0.00000,2.234680
3,1.000000,2.000000,0.00000,2.289590
4,0.587490,2.000000,0.00000,2.000000
...,...,...,...,...
298,1.000000,1.000000,0.00000,2.392724
299,1.000000,1.000000,0.83857,3.000000
300,1.000000,1.000000,0.83857,3.000000
301,1.000000,1.000000,1.00000,3.000000


In [30]:
col_is_binary = []        
col_is_numeric_cat = {}

for col in X.columns:
    feature = X[col]

    if feature.isna().sum() > 0: #ensure has nan values
        unique = feature.dropna().unique()
        unique.sort()
        unique_int_len = len(list(filter(lambda x: x.is_integer(), unique))) #checks to ensure values are ints even if cast as float                    

        if list(unique) == [0,1]:
            col_is_binary.append(col)

        elif len(unique) == unique_int_len and (unique_int_len > 2 and unique_int_len < 6): #set 
            col_is_numeric_cat[col] = unique

In [31]:
col_is_numeric_cat

{'restecg': array([0., 1., 2.]),
 'slope': array([0., 1., 2.]),
 'ca': array([0., 1., 2., 3., 4.]),
 'thal': array([0., 1., 2., 3.])}

In [79]:
for col in col_is_numeric_cat.keys():
    result[col] = result[col].round()
    possible_values = col_is_numeric_cat[col][:-2]
    print(possible_Values)
    
    #find values not matching original values in data set
    extra_value_bool = ~result[col].isin(possible_values)
    extra_values = result[col][extra_value_bool].unique()
    
    #find the closest original value and adjust it
    for o in other:
        arr = np.asarray(possible_values)
        i = (np.abs(arr - o)).argmin()
        closest = arr[i]
        result[col] = result[col].replace(o, closest)

NameError: name 'possible_Values' is not defined

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [90]:
def generate_dataset(rows, columns, percent_missing):

    size = rows * columns
    goal = round(size * percent_missing / 100)
    df = pd.DataFrame(np.random.randint(0,100,size=(rows, columns)))
    df['target'] = ((df[df.columns[-1]])/100).round()
    nan_count = 0
    
    while nan_count < goal:
        r = np.random.randint(0,rows - 1)
        c = np.random.randint(0,columns - 1) 
        df.iat[r,c]=None
        
        nan_count = df.isna().sum().sum()
    
    return df

In [94]:
df = generate_dataset(10000,100,10)
df.to_csv(directory + folder + 'test_10000r100c10m.csv', index=False)


